In [1]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import matplotlib.pyplot as plt
%matplotlib inline

/home/batman/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# Reading the saved data pickle file
df_stocks = pd.read_csv('DataFina/PNB_new_latest_concat.csv')
# df_stocks = df_stocks.iloc[:,1:]

df_stocks = df_stocks.iloc[:,[0,1,3]]
df_stocks

,news,Date,Adj Close
0,"In an interview with CNBC-TV18, Nagesh Pydah o...",2010-12-30,0.000000
1,Emkay Global Financial Services has come out w...,2010-12-27,-0.006785
2,Punjab National Bank (PNB) has support at Rs 9...,2010-12-20,-0.001863
3,"Kotak Mahindra Bank may slip to Rs 710-711, sa...",2010-12-13,0.018167
4,"PNB can head upto Rs 1100, says Technical Anal...",2010-08-12,-0.076343
5,Punjab National Bank is to announce its Q4FY10...,2010-07-12,-0.068628
6,State-run IDBI Bank has asked Kingfisher Airli...,2010-11-30,0.154141
7,State-run Punjab National Bank said on Wednesd...,2010-11-29,-0.034835
8,Borrowers will not have to wait any longer to ...,2010-11-26,-0.021001
9,Ahead of the Reserve Bank of India�s (RBI) rev...,2010-11-25,0.027921


In [3]:
df_stocks = df_stocks.dropna(axis=0, how='any')
# df_stocks = df_stocks.iloc[:,[1,5,6,8,9]]
df_stocks.shape

(607, 3)

In [4]:
# df_stocks['Adj Close'] = df_stocks['Adj Close']*100
def modify_data(s):
    
    if float(s) < 0:
        s = 0
    elif float(s) == 0:
        s = 1
    elif float(s) >0:
        s = 1
    return s
i = 0
size = df_stocks.shape[0]
while i<size:
    df_stocks.iloc[i,2]= modify_data(df_stocks.iloc[i,2])
    i = i+1

In [5]:
# selecting the prices and articles
# df_stocks = df_stocks[['Title', 'Adj Close']]

In [6]:
df_stocks.loc[:,'news'] = df_stocks.loc[:,'news'].map(lambda x: x.lstrip('.-'))

In [7]:
df_stocks

,news,Date,Adj Close
0,"In an interview with CNBC-TV18, Nagesh Pydah o...",2010-12-30,1.0
1,Emkay Global Financial Services has come out w...,2010-12-27,0.0
2,Punjab National Bank (PNB) has support at Rs 9...,2010-12-20,0.0
3,"Kotak Mahindra Bank may slip to Rs 710-711, sa...",2010-12-13,1.0
4,"PNB can head upto Rs 1100, says Technical Anal...",2010-08-12,0.0
5,Punjab National Bank is to announce its Q4FY10...,2010-07-12,0.0
6,State-run IDBI Bank has asked Kingfisher Airli...,2010-11-30,1.0
7,State-run Punjab National Bank said on Wednesd...,2010-11-29,0.0
8,Borrowers will not have to wait any longer to ...,2010-11-26,0.0
9,Ahead of the Reserve Bank of India�s (RBI) rev...,2010-11-25,1.0


In [8]:
df = df_stocks[['Adj Close']].copy()
df['Date'] = df_stocks['Date']

In [9]:
# Adding new columns to the data frame
df["compound"] = ''
df["neg"] = ''
df["neu"] = ''
df["pos"] = ''

In [10]:
df

,Adj Close,Date,compound,neg,neu,pos
0,1.0,2010-12-30,,,,
1,0.0,2010-12-27,,,,
2,0.0,2010-12-20,,,,
3,1.0,2010-12-13,,,,
4,0.0,2010-08-12,,,,
5,0.0,2010-07-12,,,,
6,1.0,2010-11-30,,,,
7,0.0,2010-11-29,,,,
8,0.0,2010-11-26,,,,
9,1.0,2010-11-25,,,,


In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sid = SentimentIntensityAnalyzer()
for date, row in df_stocks.T.iteritems():
    try:
        sentence = unicodedata.normalize('NFKD', df_stocks.loc[date, 'news'])#.encode('ascii','ignore')
        ss = sid.polarity_scores(sentence)
        df.set_value(date, 'compound', ss['compound'])
        df.set_value(date, 'neg', ss['neg'])
        df.set_value(date, 'neu', ss['neu'])
        df.set_value(date, 'pos', ss['pos'])
    except TypeError:
        print( df_stocks.loc[date, 'news'])
        print (date)
# dtest = xgb.DMatrix(np.array(x_test))

In [12]:
df.Date = df.Date

In [13]:
# train_start_date = '2013-12-30'
# train_end_date = '2017-03-17'
# test_start_date = '2017-03-18'
# test_end_date = '2018-3-16'

train = df[df['Date'] < '2017-03-17']
test = df[df['Date'] > '2017-03-18']

# train = df.iloc[train_start_date : train_end_date]
# test = df.ix[test_start_date:test_end_date]
# train.to_csv('Data/train_pnb.csv',index=None)
# test.to_csv('Data/test_pnb.csv',index=None)
test
x_test = test.iloc[:,2:]
y_test = test.iloc[:,0]

# X
test

,Adj Close,Date,compound,neg,neu,pos
537,1.0,2017-12-28,-0.5719,0.121,0.879,0
538,1.0,2017-12-19,0.3612,0.081,0.751,0.168
539,0.0,2017-12-14,0.7845,0,0.683,0.317
540,0.0,2017-12-13,0,0,1,0
541,1.0,2017-12-12,-0.7269,0.219,0.721,0.06
542,0.0,2017-05-12,0,0,1,0
543,0.0,2017-04-12,0.9042,0,0.672,0.328
544,1.0,2017-11-21,0,0,1,0
545,0.0,2017-08-11,0,0,1,0
546,1.0,2017-07-11,0.2263,0,0.881,0.119


In [14]:

print(train.shape)
train = train.dropna(axis=0, how='any')
print(train.shape)
Y = train['Adj Close']
train_new = train.drop(['Adj Close'],axis = 1)
X=train_new
X = X.iloc[:,1:]
# test_1 = test_1.drop('nLastTradedPrice',axis=1)
# X
# X.to_csv('traintatti.csv')
# Y.to_csv('traintarget.csv')
X

(554, 6)
(554, 6)


,compound,neg,neu,pos
0,0,0,1,0
1,0,0,1,0
2,0.4019,0,0.828,0.172
3,0.4939,0,0.849,0.151
4,0,0,1,0
5,0,0,1,0
6,-0.34,0.121,0.809,0.07
7,0,0,1,0
8,0.3612,0,0.921,0.079
9,0,0,1,0


In [37]:
import xgboost as xgb

# Y = np.array(Y,dtype=float)
# Y = pd.DataFrame(Y)
# Y.to_csv('tatti.csv')
# Y = Y.dropna(axis=0, how='any')
# train.shape
# X.to_csv('XG_in.csv',index=None)
# Y
dtrain = xgb.DMatrix(np.array(X), label=Y)
i = 0
row = X.shape[0]
while i<row:
    X.iloc[i,0] = float(X.iloc[i,0])
    X.iloc[i,1]= float(X.iloc[i,1])
    X.iloc[i,2]= float(X.iloc[i,2])
    X.iloc[i,3]= float(X.iloc[i,3])
    Y.iloc[i] = float(Y.iloc[i])
    i = i + 1
    print(i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [39]:
print(X)
x_test

    compound    neg    neu    pos
0          0      0      1      0
1          0      0      1      0
2     0.4019      0  0.828  0.172
3     0.4939      0  0.849  0.151
4          0      0      1      0
5          0      0      1      0
6      -0.34  0.121  0.809   0.07
7          0      0      1      0
8     0.3612      0  0.921  0.079
9          0      0      1      0
10   -0.4404  0.097  0.903      0
11         0      0      1      0
12    0.2023      0  0.935  0.065
13         0      0      1      0
14    0.2023      0  0.927  0.073
15    0.2023      0  0.948  0.052
16         0      0      1      0
17    0.4588      0   0.75   0.25
18    0.8126      0  0.802  0.198
19         0      0      1      0
20    0.2023      0  0.886  0.114
21    0.2023      0  0.935  0.065
22   -0.7096  0.138  0.862      0
23    0.0258  0.151  0.719   0.13
24         0      0      1      0
25         0      0      1      0
26   -0.3612  0.032  0.968      0
27         0      0      1      0
28         0  

,compound,neg,neu,pos
537,-0.5719,0.121,0.879,0
538,0.3612,0.081,0.751,0.168
539,0.7845,0,0.683,0.317
540,0,0,1,0
541,-0.7269,0.219,0.721,0.06
542,0,0,1,0
543,0.9042,0,0.672,0.328
544,0,0,1,0
545,0,0,1,0
546,0.2263,0,0.881,0.119


In [40]:
Y

0      1.0
1      0.0
2      0.0
3      1.0
4      0.0
5      0.0
6      1.0
7      0.0
8      0.0
9      1.0
10     1.0
11     1.0
12     1.0
13     1.0
14     0.0
15     1.0
16     1.0
17     0.0
18     0.0
19     1.0
20     1.0
21     0.0
22     0.0
23     0.0
24     1.0
25     1.0
26     1.0
27     0.0
28     0.0
29     0.0
      ... 
524    0.0
525    0.0
526    1.0
527    0.0
528    0.0
529    1.0
530    1.0
531    1.0
532    1.0
533    1.0
534    0.0
535    0.0
536    0.0
547    0.0
562    0.0
563    1.0
577    0.0
578    1.0
579    0.0
580    0.0
581    1.0
582    1.0
583    0.0
584    0.0
587    0.0
588    1.0
589    0.0
590    0.0
591    0.0
594    0.0
Name: Adj Close, Length: 554, dtype: float64

In [45]:

import xgboost as xgb
import operator

dtrain = xgb.DMatrix(np.array(X), label=Y)
#dval=xgb.DMatrix(np.array(x_val),label=y_val)
dtest = xgb.DMatrix(np.array(x_test))

xgb_params = {
    'objective': 'multi:softmax',
    'booster': 'gbtree',
    'eval_metric': 'error',
    'lambda': 2,
    'alpha': 0.02,
    'subsample': 0.8,
    #'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'colsample_bytree': 0.8,
#     'min_child_weight': 4,
    'silent': 1,
    'num_class':3,
    'max_depth':8,
    'min_child_weight':50,
           'eta':0.2,
            'seed':88888
}
num_round=2000
#watchlist=[(dtrain,'train'),(dval,'eval')]

#gbdt = xgb.train(xgb_params, dtrain,num_round,watchlist,early_stopping_rounds=50)
gbdt = xgb.train(xgb_params, dtrain,num_round)
# def ceate_feature_map(features):
#     outfile = open('xgb.fmap', 'w')
#     i = 0
#     for feat in features:
#         outfile.write('{0}\t{1}\tq\n'.format(i, feat))
#         i = i + 1

#     outfile.close()
# ceate_feature_map(list(train.columns))
# print(train.columns)
# xgb.plot_importance(gbdt)
# # gbdt=gbdt
# importance = gbdt.get_fscore(fmap='xgb.fmap')
# importance = sorted(importance.items(), key=operator.itemgetter(1))
# print(importance)
# df = pd.DataFrame(importance, columns=['feature', 'fscore'])
# df['fscore'] = df['fscore'] / df['fscore'].sum()
# #print df
# plt.figure()
# df.plot()
# df.plot(kind='barh', x='feature', y='fscore', legend=False)
# plt.title('XGBoost Feature Importance')
# plt.xlabel('relative importance')
# plt.gcf().savefig('feature_importance_xgb.png')

In [46]:
pred = gbdt.predict(dtest)
# pred = pred.astype(int) 
# size = len(pred)
# i = 0
# while i <size:
#     if pred[i]<0.5:
#         pred[i]= 0
#     else:
#         pred[i]=1
#     i = i+1
print(pred)
y_test
# pred = pred.astype(int) 

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.
  0.  1.  1.  1.  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.
  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.]


537    1.0
538    1.0
539    0.0
540    0.0
541    1.0
542    0.0
543    0.0
544    1.0
545    0.0
546    1.0
548    1.0
549    0.0
550    0.0
551    1.0
552    1.0
553    0.0
554    0.0
555    1.0
556    1.0
557    1.0
558    1.0
559    0.0
560    1.0
561    0.0
564    0.0
565    1.0
566    1.0
567    1.0
568    1.0
569    1.0
570    0.0
571    0.0
572    1.0
573    0.0
574    0.0
575    0.0
576    1.0
585    1.0
586    0.0
592    1.0
593    0.0
595    0.0
596    0.0
597    1.0
598    0.0
599    0.0
600    1.0
601    1.0
602    1.0
603    1.0
604    0.0
605    0.0
606    1.0
Name: Adj Close, dtype: float64

In [47]:
# real = pd.read_csv('Ans/test.csv')
# real
# real_1 = real.iloc[:,1]
# real_2 = real.iloc[:,3]
# real_1 = pd.read_csv('Ans/test.csv')
# real_2 = pd.read_csv('Ans/pred.csv')

In [48]:
size = len(pred)
i = 0
count = 0
while i < size:
    if pred[i]==y_test.iloc[i]:
        count = count + 1
    i = i +1
accuracy = count/size *100
print(accuracy)

56.60377358490566


In [ ]:
pred

In [ ]:
import pickle

In [50]:
# s = pickle.dumps(gbdt)
# clf2 = pickle.loads(s)
# pred1= clf2.predict(dtest)
# pred1
filename = 'pnb_model_better.pkl'
pickle.dump(gbdt, open(filename, 'wb'),protocol=2)

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(dtest)
print(result)

In [52]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pickle
import unicodedata
article = ['Atrocious Service! really HATED']
sid = SentimentIntensityAnalyzer()
# # for date, row in df_stocks.T.iteritems():
# #     try:
# sentence = unicodedata.normalize('NFKD', article)#.encode('ascii','ignore')
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.predict(dtest)
# print(result)
def mohan():
    ss = sid.polarity_scores(article[0])
    answ = {
        'index': 0,
        'compound':ss['compound'],

        'neg':ss['neg'],

        'neu':ss['neu'],

        'pos':ss['pos'],
    }
    lol = []
    lol.append(answ)
    answ = pd.DataFrame(lol)
    print(answ)
    answ = answ.iloc[:,[0,2,3,4]]
    check = xgb.DMatrix(np.array(answ))
    result = loaded_model.predict(check)
    print(result)
    
mohan()
# answ
# ss['compound']

   compound  index    neg    neu  pos
0   -0.7592      0  0.648  0.352  0.0
[ 0.]


In [ ]:
from sklearn.cross_validation import KFold
import xgboost as xgb
from sklearn import  metrics
a = np.array([])
kfolds = KFold(df_stocks.shape[0], n_folds=6)
for dev_index, val_index in kfolds:
    dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    dtrain = xgb.DMatrix(dev_X,label = dev_y)
    dtest = xgb.DMatrix(val_X)
    bst = xgb.train( plst,dtrain, num_rounds)
    ypred_bst = bst.predict(dtest,ntree_limit=bst.best_iteration)
    score = metrics.confusion_matrix(val_y, ypred_bst)
    res = (score[0][0]+score[1][1])*1.0/sum(sum(score))
    a = np.append(a,[res])
    print ("Accuracy = %.7f" % (res))
print ("Overall Mean Accuracy = %.7f" % (np.mean(a)))

In [ ]:
print(Accuracy)